In [1]:
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, f1_score
from sklearn.preprocessing import Normalizer
import tensorflow as tf
from tqdm import tqdm

In [2]:
print(tf.__version__)

0.12.0


In [3]:
from data import get_data

In [4]:
X_train, y_train, X_test, y_test = get_data(one_hot=True)

In [5]:
mean_of_train = np.mean(X_train)
std_of_train = np.std(X_train)
print(mean_of_train, std_of_train)

115.481949803 65.2925375088


In [6]:
X_train = (X_train - mean_of_train) / std_of_train
X_test = (X_test - mean_of_train) / std_of_train

In [7]:
train_len, width, height, channel = X_train.shape
test_len = X_test.shape[0]
print("""
训练数据数量：{}
高度:{}
宽度：{}
测试数据数量：{}
""".format(train_len, width, height, channel, test_len))


训练数据数量：20000
高度:128
宽度：128
测试数据数量：3



In [8]:
# 学习率
learning_rate = 0.003
# 迭代次数（批次）
n_epoch = 10
# 批次大小
# 可能需要调小
batch_size = 32
# 露点
leakiness=0.1
# 输出大小
target_size = y_train.shape[1]
print('target_size is', target_size)

target_size is 2


In [9]:
tf.set_random_seed(0)

In [10]:
# 输入占位符
X = tf.placeholder(tf.float32, [batch_size, width, height, channel])
# 输出占位符
y = tf.placeholder(tf.float32, [batch_size, target_size])

In [11]:
def conv(input_layer, output_size, pitch_shape, name, strides=[1, 1, 1, 1], padding='VALID'):
    """卷积层"""
    with tf.variable_scope(name):
        shape = [
            pitch_shape[0],
            pitch_shape[1],
            int(input_layer.get_shape()[-1]),
            output_size
        ]
        kernel = tf.Variable(tf.random_normal(shape, stddev=np.sqrt(2.0 / (shape[0] + shape[1] + shape[3]))))
        bias = tf.Variable(tf.zeros([shape[-1]]))
        conv = tf.nn.bias_add(
            tf.nn.conv2d(
                input_layer, kernel, strides=strides, padding=padding
            ),
            bias
        )
        print(name, conv.get_shape())
        return conv

In [12]:
def relu(x, leakiness=0.0):
    """Relu, with optional leaky support.
    borrow from https://github.com/tensorflow/models/blob/master/resnet/resnet_main.py
    """
    return tf.select(tf.less(x, 0.0), leakiness * x, x, name='leaky_relu')

In [13]:
def max_pool(input_layer, name, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID'):
    """最大池化"""
    mp = tf.nn.max_pool(input_layer, ksize=ksize, strides=strides, padding=padding)
    print(name, 'output shape is', mp.get_shape())
    return mp

In [14]:
model = X

In [15]:
model = relu(conv(model, 64, (5, 5), 'conv_1', strides=(1, 2, 2, 1)), leakiness=leakiness)

conv_1 (32, 62, 62, 64)


In [16]:
model = max_pool(model, 'max_pool_1')

max_pool_1 output shape is (32, 31, 31, 64)


In [17]:
model = relu(conv(model, 128, (5, 5), 'conv_3', strides=(1, 2, 2, 1)), leakiness=leakiness)

conv_3 (32, 14, 14, 128)


In [18]:
model = max_pool(model, 'max_pool_2')

max_pool_2 output shape is (32, 7, 7, 128)


In [19]:
model = relu(conv(model, 256, (3, 3), 'conv_5'), leakiness=leakiness)

conv_5 (32, 5, 5, 256)


In [20]:
model = max_pool(model, 'max_pool_3')

max_pool_3 output shape is (32, 2, 2, 256)


In [21]:
model = tf.reshape(model, [batch_size, -1])

In [22]:
print(model.get_shape())
dim = int(model.get_shape()[1])
print('dim is {}'.format(dim))

(32, 1024)
dim is 1024


In [23]:
def full_connect(input_layer, output_size, name, reuse=True):
    """全连接层"""
    with tf.variable_scope(name):
        shape = [int(input_layer.get_shape()[1]), output_size]
        weight = tf.Variable(
            tf.random_normal(shape, stddev=np.sqrt(2.0 / (shape[0] + shape[1])))
        )
        bias = tf.Variable(tf.zeros([shape[-1]]))
        fc = tf.nn.bias_add(
            tf.matmul(input_layer, weight),
            bias
        )
        print(name, fc.get_shape())
        return fc

In [24]:
def full_connect_with_bn(input_layer, output_size, name, reuse=True):
    """全连接层"""
    with tf.variable_scope(name):
        shape = [int(input_layer.get_shape()[1]), output_size]
        weight = tf.Variable(
            tf.random_normal(shape, stddev=np.sqrt(2.0 / (shape[0] + shape[1])))
        )
        beta = tf.Variable(tf.zeros([shape[-1]]))
        scale = tf.Variable(tf.zeros([shape[-1]]))
        z = tf.matmul(input_layer, weight)
        # 一个很小的数，避免可能的除0错
        epsilon = 1e-08
        # 计算mean和variance
        batch_mean, batch_var = tf.nn.moments(z, [0])
        fc = tf.nn.batch_normalization(z, batch_mean, batch_var, beta, scale, epsilon)
        print(name, fc.get_shape())
        return fc

In [25]:
model = relu(full_connect_with_bn(model, 1024, 'fc_1'), leakiness=leakiness)

fc_1 (32, 1024)


In [26]:
model = full_connect(model, target_size, 'fc_2')

fc_2 (32, 2)


In [27]:
pred = model

In [28]:
cost = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(
        pred, y
    )
)

In [29]:
params = tf.trainable_variables()

In [30]:
gradients = tf.gradients(cost, params)

In [31]:
clipped_gradients, norm = tf.clip_by_global_norm(
    gradients,
    5.0
)

In [32]:
opt = tf.train.AdamOptimizer(learning_rate=learning_rate)

In [33]:
train_step = opt.apply_gradients(zip(clipped_gradients, params))

In [34]:
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [35]:
init = tf.global_variables_initializer()

In [36]:
def batch_flow(inputs, targets, batch_size):
    """流动数据流"""
    flowed = 0
    total = len(inputs)
    while True:
        X_ret = []
        y_ret = []
        for i in range(total):
            X_ret.append(inputs[i])
            y_ret.append([targets[i]])
            if len(X_ret) == batch_size:
                flowed += batch_size
                X, y = np.array(X_ret), np.array(y_ret)
                y = y.reshape([batch_size, -1])
                yield X, y
                X_ret = []
                y_ret = []
            if flowed >= total:
                break
        if flowed >= total:
            break

In [37]:
for batch_x, batch_y in batch_flow(X_train, y_train, batch_size):
    print(batch_x.shape, batch_y.shape)
    break

(32, 128, 128, 3) (32, 2)


In [38]:
with tf.Session() as sess:
    sess.run(init)
    total = None
    for epoch in range(n_epoch):
        costs = []
        accs = []
        for batch_x, batch_y in tqdm(batch_flow(X_train, y_train, batch_size), total=total):
            _, c, acc = sess.run([train_step, cost, accuracy], feed_dict={X: batch_x, y: batch_y})
            costs.append(c)
            accs.append(acc)
        print('epoch: {}, loss: {:.4f}, acc: {:.4f}'.format(epoch, np.mean(costs), np.mean(accs)))
        if total is None:
            total = len(costs)
    print('calculate test accuracy')
    costs = []
    accs = []
    for batch_x, batch_y in tqdm(batch_flow(X_test, y_test, batch_size)):
        c, acc = sess.run([cost, accuracy], feed_dict={X: batch_x, y: batch_y})
        costs.append(c)
        accs.append(acc)
    print('test loss: {:.4f}, acc: {:.4f}'.format(np.mean(costs), np.mean(accs)))
    print('Done')

625it [00:12, 48.43it/s]0:00,  2.86it/s]
  1%|          | 5/625 [00:00<00:12, 48.79it/s]

epoch: 0, loss: 0.5597, acc: 0.7091


  1%|          | 5/625 [00:00<00:12, 48.16it/s]

epoch: 1, loss: 0.4260, acc: 0.8004


  1%|          | 5/625 [00:00<00:12, 48.53it/s]

...

epoch: 8, loss: 0.0673, acc: 0.9739


100%|██████████| 625/625 [00:12<00:00, 51.40it/s] 3%|▎         | 16/625 [00:00<00:12, 49.48it/s]
9it [00:00, 88.63it/s]

epoch: 9, loss: 0.0541, acc: 0.9796
calculate test accuracy


157it [00:01, 96.74it/s]9it [00:00, 91.21it/s]

test loss: 0.9241, acc: 0.8014
Done


train:
loss: 0.0541, acc: 0.9796

test:
loss: 0.9241, acc: 0.8014